## To Do
- find bridge geometry files
- convert lat/lon to WGS84
- convert timestamp and try to separate "trips"

## Future
- think about downloading moving pandas?

In [26]:
import pandas as pd
import json
import geopandas as gpd
import folium

In [20]:
with open("location_data/Records.json", 'rb') as f:
    record_json = json.loads(f.read())

In [21]:
record_json['locations'][0]

{'latitudeE7': 476141570,
 'longitudeE7': -1223229821,
 'accuracy': 30,
 'deviceTag': 532271696,
 'deviceDesignation': 'PRIMARY',
 'timestamp': '2017-12-21T03:26:02.032Z'}

In [22]:
records_df = pd.DataFrame(record_json['locations'])
records_df.head(3)

,latitudeE7,longitudeE7,accuracy,deviceTag,deviceDesignation,timestamp,source,velocity,heading,altitude,verticalAccuracy,platformType,serverTimestamp,deviceTimestamp,batteryCharging,formFactor
0,476141570,-1223229821,30,532271696,PRIMARY,2017-12-21T03:26:02.032Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,476141557,-1223231129,41,532271696,PRIMARY,2017-12-21T03:38:47.019Z,VISIT_DEPARTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,476152483,-1223254265,10,532271696,PRIMARY,2017-12-21T03:39:34.439Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
records_df['latitude'] = records_df['latitudeE7'].apply(lambda x: x/10000000)
records_df['longitude'] = records_df['longitudeE7'].apply(lambda x: x/10000000)
records_df.head(3)

,latitudeE7,longitudeE7,accuracy,deviceTag,deviceDesignation,timestamp,source,velocity,heading,altitude,verticalAccuracy,platformType,serverTimestamp,deviceTimestamp,batteryCharging,formFactor,latitude,longitude
0,476141570,-1223229821,30,532271696,PRIMARY,2017-12-21T03:26:02.032Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.614157,-122.322982
1,476141557,-1223231129,41,532271696,PRIMARY,2017-12-21T03:38:47.019Z,VISIT_DEPARTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.614156,-122.323113
2,476152483,-1223254265,10,532271696,PRIMARY,2017-12-21T03:39:34.439Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.615248,-122.325427


In [25]:
geometry = gpd.points_from_xy(records_df.longitude, records_df.latitude, crs="EPSG:4326")
records_gpf = gpd.GeoDataFrame(records_df, crs="EPSG:4326", geometry=geometry)

In [29]:
records_gpf.shape

(40384, 19)

## Import bridge buffer

In [28]:
bridge_buffers = gpd.read_file('gis_data/portland_river_bridge_buffer.geojson')

In [30]:
records_on_bridge = gpd.overlay(records_gpf, bridge_buffers, how='intersection', keep_geom_type=True, make_valid=True)

In [35]:
records_on_bridge.shape

(224, 25)

In [36]:
m = folium.Map(location=[45.5236, -122.6750], zoom_start=13)
folium.GeoJson(data=records_on_bridge['geometry']
# , tooltip=folium.features.GeoJsonTooltip(fields=['NAME'])
    ).add_to(m)

m